# Visualising geomagnetic data

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from viresclient import SwarmRequest

## Fetching & loading INTERMAGNET data

We use VirES to access the ground observatory [(see here for more)](https://swarm.magneticearth.org/notebooks/04c2_geomag-ground-data-vires)

NB: Data is provided in the NEC (North, East, Centre) geocentrically-defined frame, in contrast to [INTERMAGNET data](https://intermagnet.github.io/) from other sources where the geodetic frame is used. This results in a small rotation in the North (X) and Centre (Z) vectors and a slightly different "latitude" for the observatory location.

In [ ]:
def fetch_obs_data_for_year(observatory="ESK", year=2003, cadence="M"):
    """Fetch a year of observatory data from VirES, at minute (M) or hour (H) cadence
    
    Args:
        observatory (str): 3-letter IAGA code
        year (int): Chosen year
        cadence (str): "M" for minute, or "H" for hour
        
    Returns:
        DataFrame
    """
    request = SwarmRequest()
    request.set_collection(f"SW_OPER_AUX_OBS{cadence}2_:{observatory}", verbose=False)
    request.set_products(measurements=["B_NEC", "IAGA_code"])
    data = request.get_between(
        dt.datetime(year, 1, 1),
        dt.datetime(year+1, 1, 1)
    )
    df = data.as_dataframe(expand=True).drop(columns="Spacecraft")
    df = df.rename(columns={f"B_NEC_{i}": j for i, j in zip("NEC", "XYZ")})
    # ds = data.as_xarray().drop("Spacecraft")
    return df

obs_min = fetch_obs_data_for_year(
    observatory="ESK",
    year=2003,
    cadence="M"
)
obs_min.head()

## Plotting 1-minute data and their hourly means

In [ ]:
def plot_subset(start_date, end_date, hourly_mean=False, show_annual_mean=False, df=obs_min):
    """Configurably plot a subset of the data
    
    Args:
        start_date (datetime)
        end_date (datetime)
        hourly_mean (bool): Evaluate and plot the hourly means instead
        show_annual_mean (bool): Show offset from annual mean
        
    Returns: 
    """
    annual_mean = df.resample("1y").mean()
    df = df.loc[start_date:end_date]
    observatory = obs_min["IAGA_code"][0]
    title = f"Minute data from {observatory}"
    if hourly_mean:
        df = df.resample("1h").mean()
        title += ": averaged over each hour"
    fig, axes = plt.subplots(nrows=3, figsize=(10, 7), sharex=True)
    for i, cpt in enumerate("XYZ"):
        if show_annual_mean:
            axes[i].fill_between(df.index, df[cpt], annual_mean[cpt])
        else:
            df[cpt].plot(ax=axes[i])
            axes[i].plot(df[cpt])
        axes[i].set_ylabel(f"{cpt} (nT)")
        axes[i].grid()
    fig.suptitle(title)
    axes[2].set_xlabel("Date")
    fig.tight_layout()
    return fig

def make_widgets():
    """Use ipywidgets to interact with plot_subset"""
    mini, maxi = str(obs_min.index.min().date()), str(obs_min.index.max().date())
    start_date = widgets.DatePicker(
        value=dt.datetime(2003, 1, 1),
        description='Start Date',
    )
    end_date = widgets.DatePicker(
        value=dt.datetime(2003, 2, 1),
        description='End Date',
    )
    hourly_mean = widgets.Checkbox(
        value=True,
        description='Hourly mean',
    )
    annual_mean = widgets.Checkbox(
        value=True,
        description='Show annual mean',
    )
    return widgets.VBox(
        [widgets.Label(f"Select dates within range: {mini}, {maxi}"),
         widgets.HBox([start_date, end_date, widgets.VBox([hourly_mean, annual_mean])]),
         widgets.interactive_output(
             plot_subset,
             {'start_date': start_date, 'end_date': end_date, 'hourly_mean': hourly_mean, 'show_annual_mean': annual_mean}
         )]
    )

make_widgets()